chilly

In [3]:
import cv2
import numpy as np
import imutils
from skimage.metrics import structural_similarity as ssim

CHILI_MIN_ASPECT_RATIO = 0.2  
CHILI_MAX_ASPECT_RATIO = 5.0
CHILI_MIN_AREA = 800
AFFECTED_AREA_THRESHOLD = 0.2
SIMILARITY_THRESHOLD = 0.4  

def resize_image(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def calculate_ssim(image1, image2):
    """Calculate SSIM similarity score between two images."""
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    
    h, w = min(image1_gray.shape[0], image2_gray.shape[0]), min(image1_gray.shape[1], image2_gray.shape[1])
    image1_gray = cv2.resize(image1_gray, (w, h))
    image2_gray = cv2.resize(image2_gray, (w, h))

    score, _ = ssim(image1_gray, image2_gray, full=True)
    return score

def detect_chili_quality(image_path, reference_path):
    """Detect chili quality, compare with reference, and highlight affected areas."""
    image = cv2.imread(image_path)
    reference_image = cv2.imread(reference_path)
    
    if image is None or reference_image is None:
        print("Error: One or both input images not found or failed to load.")
        return
    
    print("Input and reference images loaded successfully.")

    cv2.imshow("Original Image", resize_image(image, 800, 600))
    cv2.imshow("Reference Image", resize_image(reference_image, 800, 600))

    # To Adjust brightness and contrast
    alpha = 2.0
    beta = 0
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    hsv = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)

    # Mask for red chilies
    lower_red1 = np.array([0, 30, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 30, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    chili_mask = mask1 + mask2

    # Morphological operations
    kernel = np.ones((5, 5), np.uint8)
    chili_mask = cv2.morphologyEx(chili_mask, cv2.MORPH_CLOSE, kernel)
    chili_mask = cv2.morphologyEx(chili_mask, cv2.MORPH_OPEN, kernel)
    chili_mask = cv2.dilate(chili_mask, None, iterations=1)

    # Find contours
    cnts = cv2.findContours(chili_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    output_image = adjusted_image.copy()

    # HSV mask for affected areas
    lower_affected = np.array([5, 30, 90])
    upper_affected = np.array([50, 180, 270])
    mask_affected = cv2.inRange(hsv, lower_affected, upper_affected)

    total_chilies = 0
    bad_quality_chilies = 0

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > CHILI_MIN_AREA:
            x, y, w, h = cv2.boundingRect(c)
            aspect_ratio = float(h) / w if w > 0 else 0

            if CHILI_MIN_ASPECT_RATIO <= aspect_ratio <= CHILI_MAX_ASPECT_RATIO:
                total_chilies += 1

                # Extract chili ROI for similarity check
                chili_roi = adjusted_image[y:y+h, x:x+w]

                # To calculate similarity with reference image
                similarity_score = calculate_ssim(chili_roi, reference_image)

                # Mask and detect affected areas
                single_chili_mask = np.zeros(chili_mask.shape, dtype=np.uint8)
                cv2.drawContours(single_chili_mask, [c], -1, 255, thickness=cv2.FILLED)
                affected_chili = cv2.bitwise_and(mask_affected, mask_affected, mask=single_chili_mask)

                # To Find contours in affected area
                affected_contours = cv2.findContours(affected_chili, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                affected_contours = imutils.grab_contours(affected_contours)

                # yellow contours on affected regions
                for affected_contour in affected_contours:
                    cv2.drawContours(output_image, [affected_contour], -1, (0, 255, 255), 2)  # Yellow for affected regions

                affected_area = cv2.countNonZero(affected_chili)
                affected_ratio = affected_area / area if area > 0 else 0

                # Decide quality based on SSIM and affected area
                if affected_ratio > AFFECTED_AREA_THRESHOLD or similarity_score < SIMILARITY_THRESHOLD:
                    bad_quality_chilies += 1
                    label = "Bad Quality Chili"
                    color = (0, 0, 255)  # Red for bad quality
                else:
                    label = "Good Quality Chili"
                    color = (0, 255, 0)  # Green for good quality

                # Draw bounding box and labels
                cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                cv2.putText(output_image, f"Similarity: {similarity_score:.2f}", (x, y + h + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Final grading summary
    print(f"Total Chilies: {total_chilies}")
    print(f"Bad Quality Chilies: {bad_quality_chilies}")
    print(f"Good Quality Chilies: {total_chilies - bad_quality_chilies}")

    # Display results
    grading_text = f"Total: {total_chilies}, Bad: {bad_quality_chilies}, Good: {total_chilies - bad_quality_chilies}"
    cv2.putText(output_image, grading_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
    
    cv2.imshow("Final Output", resize_image(output_image, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Paths to input and reference images
image_path = r'C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\Chilly\IMG_20241102_122_14.jpg'
reference_path = r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\image dataset\chilli\single\IMG_20241102_121300.jpg"

detect_chili_quality(image_path, reference_path)


Input and reference images loaded successfully.
Total Chilies: 6
Bad Quality Chilies: 0
Good Quality Chilies: 6


pepper

In [4]:
import cv2
import numpy as np
import imutils
from sklearn.cluster import KMeans

# Constants
PEPPER_MIN_AREA = 1000
PEPPER_MIN_ASPECT_RATIO = 0.2
PEPPER_MAX_ASPECT_RATIO = 5.0
AFFECTED_RATIO_THRESHOLD = 0.15  # Raised to reduce false positives

# HSV thresholds for bad peppers (brown/yellowish)
BAD_HUE_MIN = 10    # Brown/yellow hues typically start here
BAD_HUE_MAX = 40    # Up to yellow/orange
BAD_SAT_MIN = 50    # Moderate saturation for brown/yellow
BAD_VAL_MIN = 50    # Minimum brightness to exclude very dark areas

def resize_image(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def get_adaptive_hsv_range(image):
    """Determine HSV range dynamically using K-Means clustering."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    pixels = hsv.reshape(-1, 3)
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(pixels)
    centers = kmeans.cluster_centers_
    pepper_cluster = sorted(centers, key=lambda x: x[2])[1]  # Middle Value cluster
    h_range, s_range, v_range = 180, 255, 73
    lower_pepper = np.array([max(0, pepper_cluster[0] - h_range),
                            max(0, pepper_cluster[1] - s_range),
                            max(0, pepper_cluster[2] - v_range)], dtype=np.uint8)
    upper_pepper = np.array([min(180, pepper_cluster[0] + h_range),
                            min(255, pepper_cluster[1] + s_range),
                            min(255, pepper_cluster[2] + v_range)], dtype=np.uint8)
    return lower_pepper, upper_pepper

def detect_pepper_quality(image_path, rf):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to load image.")
        return
    print("Image loaded successfully.")

    # Adjust brightness and contrast
    alpha = 2.0
    beta = 0
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    # Get adaptive HSV range
    lower_pepper, upper_pepper = get_adaptive_hsv_range(adjusted_image)
    print(f"Adaptive HSV Range: Lower={lower_pepper}, Upper={upper_pepper}")
    
    # Convert to HSV and apply mask
    hsv = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)
    pepper_mask = cv2.inRange(hsv, lower_pepper, upper_pepper)

    # Morphological operations
    kernel = np.ones((7, 7), np.uint8)
    pepper_mask = cv2.morphologyEx(pepper_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    pepper_mask = cv2.morphologyEx(pepper_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours
    cnts = cv2.findContours(pepper_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # Prepare output image
    output = adjusted_image.copy()
    total_peppers = 0
    bad_quality_peppers = 0

    # Filter contours
    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area < PEPPER_MIN_AREA:
            continue

        x, y, w, h = cv2.boundingRect(c)
        aspect_ratio = float(h) / w if w > 0 else 0

        if PEPPER_MIN_ASPECT_RATIO <= aspect_ratio <= PEPPER_MAX_ASPECT_RATIO:
            total_peppers += 1

            # Create mask for this pepper
            single_pepper_mask = np.zeros_like(pepper_mask)
            cv2.drawContours(single_pepper_mask, [c], -1, 255, thickness=cv2.FILLED)
            single_pepper_mask_cropped = single_pepper_mask[y:y+h, x:x+w]

            # Edge detection
            roi = adjusted_image[y:y+h, x:x+w]
            roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            roi_edges = cv2.Canny(roi_gray, 50, 150)
            affected_region = cv2.bitwise_and(roi_edges, single_pepper_mask_cropped)

            # Calculate affected area ratio
            affected_pixels = cv2.countNonZero(affected_region)
            total_pepper_pixels = cv2.countNonZero(single_pepper_mask_cropped)
            affected_ratio = affected_pixels / total_pepper_pixels if total_pepper_pixels > 0 else 0

            # Color analysis
            mean_hsv = cv2.mean(hsv[y:y+h, x:x+w], mask=single_pepper_mask_cropped)[:3]  # Mean HSV
            hue, sat, val = mean_hsv

            # Classify quality: Brown/yellowish = bad, or excessive edges
            is_brown_yellow = (BAD_HUE_MIN <= hue <= BAD_HUE_MAX and sat >= BAD_SAT_MIN and val >= BAD_VAL_MIN)
            if is_brown_yellow or affected_ratio > AFFECTED_RATIO_THRESHOLD:
                label = "Bad Quality"
                color = (0, 0, 255)
                bad_quality_peppers += 1
            else:
                label = "Good Quality"
                color = (0, 255, 0)

            # Draw bounding box and labels
            cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
            cv2.putText(output, label, (x, max(y - 10, 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            cv2.putText(output, f"Ratio: {affected_ratio:.2f}", (x, y + h + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(output, f"H:{hue:.1f} S:{sat:.1f} V:{val:.1f}", (x, y + h + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display summary
    summary = f"Total: {total_peppers}, Bad: {bad_quality_peppers}, Good: {total_peppers - bad_quality_peppers}"
    cv2.putText(output, summary, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    print(summary)

    # Show results
    cv2.imshow("Pepper Quality Detection", resize_image(output, 800, 600))
    cv2.imshow("Pepper Mask", resize_image(pepper_mask, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function
image_path = r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\Pepper\IMG_20241102_125400_0.jpg"
reference_path = r"C:\Users\vidya\Downloads\IMG_20250216_202203_1.jpg"
detect_pepper_quality(image_path, reference_path)

Image loaded successfully.
Adaptive HSV Range: Lower=[0 0 9], Upper=[180 255 155]
Total: 6, Bad: 3, Good: 3


cardamom

In [15]:
import cv2
import numpy as np
import imutils
from sklearn.cluster import KMeans

# Constants
CARDAMOM_MIN_AREA = 10
CARDAMOM_MIN_ASPECT_RATIO = 0.05
CARDAMOM_MAX_ASPECT_RATIO = 20.0
AFFECTED_RATIO_THRESHOLD = 0.20
SHAPE_MATCH_THRESHOLD = 5.0
resize_scale = 0.5

# HSV thresholds for bad cardamom
BAD_HUE_MIN = 25  # Raised to exclude green (e.g., 20.7)
BAD_HUE_MAX = 35
BAD_SAT_MIN = 50
BAD_VAL_MIN = 50
CRACK_VAL_MAX = 50
CRACK_RATIO_THRESHOLD = 0.01

def resize_image(image, scale):
    width = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def normalize_contour(contour, target_size=(100, 100)):
    x, y, w, h = cv2.boundingRect(contour)
    scale_x, scale_y = target_size[0] / w, target_size[1] / h
    return np.array([[[int(pt[0][0] * scale_x), int(pt[0][1] * scale_y)]] for pt in contour])

def load_reference_contour(reference_image_path):
    reference_image = cv2.imread(reference_image_path, 0)
    if reference_image is None:
        print("Error: Reference image not found.")
        return None

    _, thresholded_ref = cv2.threshold(reference_image, 127, 255, cv2.THRESH_BINARY)
    ref_contours, _ = cv2.findContours(thresholded_ref, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(ref_contours) > 0:
        epsilon = 0.03 * cv2.arcLength(ref_contours[0], True)
        ref_contour = cv2.approxPolyDP(ref_contours[0], epsilon, True)
        return normalize_contour(ref_contour)
    else:
        print("No contours found in the reference image.")
        return None

def get_adaptive_hsv_range(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    pixels = hsv.reshape(-1, 3)
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(pixels)
    centers = kmeans.cluster_centers_
    cardamom_cluster = sorted(centers, key=lambda x: x[2])[1]
    h_range, s_range, v_range = 50, 100, 100
    lower_cardamom = np.array([max(0, cardamom_cluster[0] - h_range),
                               max(0, cardamom_cluster[1] - s_range),
                               max(0, cardamom_cluster[2] - v_range)], dtype=np.uint8)
    upper_cardamom = np.array([min(180, cardamom_cluster[0] + h_range),
                               min(255, cardamom_cluster[1] + s_range),
                               min(255, cardamom_cluster[2] + v_range)], dtype=np.uint8)
    return lower_cardamom, upper_cardamom

def detect_cardamom(image_path, ref_contour):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not load image.")
        return

    height, width = image.shape[:2]
    image_resized = cv2.resize(image, (width // 2, height // 2))
    adjusted_image = image_resized

    lower_cardamom, upper_cardamom = get_adaptive_hsv_range(adjusted_image)
    print(f"Adaptive HSV Range: Lower={lower_cardamom}, Upper={upper_cardamom}")

    hsv = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_cardamom, upper_cardamom)

    kernel = np.ones((10, 10), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    print(f"Number of contours found: {len(contours)}")
    output = adjusted_image.copy()
    total_cardamom = 0
    bad_quality_cardamom = 0

    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        print(f"Contour {i} - Area: {area}")
        if area < CARDAMOM_MIN_AREA:
            print(f"Contour {i} skipped: Area {area} < {CARDAMOM_MIN_AREA}")
            continue

        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(h) / w if w > 0 else 0
        print(f"Contour {i} - Aspect Ratio: {aspect_ratio}")

        if not (CARDAMOM_MIN_ASPECT_RATIO <= aspect_ratio <= CARDAMOM_MAX_ASPECT_RATIO):
            print(f"Contour {i} skipped: Aspect Ratio {aspect_ratio} out of range")
            continue

        epsilon = 0.03 * cv2.arcLength(contour, True)
        approx_contour = cv2.approxPolyDP(contour, epsilon, True)
        approx_contour = normalize_contour(approx_contour)

        match_score = cv2.matchShapes(ref_contour, approx_contour, cv2.CONTOURS_MATCH_I1, 0.0)
        print(f"Contour {i} - Match Score: {match_score}")

        total_cardamom += 1

        single_cardamom_mask = np.zeros_like(mask)
        cv2.drawContours(single_cardamom_mask, [contour], -1, 255, thickness=cv2.FILLED)
        single_cardamom_mask_cropped = single_cardamom_mask[y:y+h, x:x+w]

        # Edge detection
        roi = adjusted_image[y:y+h, x:x+w]
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi_edges = cv2.Canny(roi_gray, 50, 150)
        affected_region = cv2.bitwise_and(roi_edges, single_cardamom_mask_cropped)
        affected_pixels = cv2.countNonZero(affected_region)
        total_pixels = cv2.countNonZero(single_cardamom_mask_cropped)
        affected_ratio = affected_pixels / total_pixels if total_pixels > 0 else 0

        # Color analysis
        mean_hsv = cv2.mean(hsv[y:y+h, x:x+w], mask=single_cardamom_mask_cropped)[:3]
        hue, sat, val = mean_hsv

        # Crack detection
        roi_hsv = hsv[y:y+h, x:x+w]
        crack_mask = cv2.inRange(roi_hsv, (0, 0, 0), (180, 255, CRACK_VAL_MAX))
        crack_mask = cv2.bitwise_and(crack_mask, single_cardamom_mask_cropped)
        kernel_dilate = np.ones((3, 3), np.uint8)
        crack_mask = cv2.dilate(crack_mask, kernel_dilate, iterations=1)
        crack_pixels = cv2.countNonZero(crack_mask)
        crack_ratio = crack_pixels / total_pixels if total_pixels > 0 else 0

        # Classify quality
        is_brown_yellow = (BAD_HUE_MIN <= hue <= BAD_HUE_MAX and sat >= BAD_SAT_MIN and val >= BAD_VAL_MIN)
        has_cracks = crack_ratio > CRACK_RATIO_THRESHOLD
        if is_brown_yellow or has_cracks:
            label = "Bad Cardamom"
            color = (0, 0, 255)
            bad_quality_cardamom += 1
        else:
            label = "Good Cardamom"
            color = (0, 255, 0)

        print(f"Contour {i} - HSV: H:{hue:.1f} S:{sat:.1f} V:{val:.1f}")
        print(f"Contour {i} - Edge Ratio: {affected_ratio:.2f}")
        print(f"Contour {i} - Crack Ratio: {crack_ratio:.2f}")
        print(f"Contour {i} - Brown/Yellow: {is_brown_yellow}, Has Cracks: {has_cracks}")
        print(f"Contour {i} - Labeled as: {label}")

        cv2.rectangle(output, (x, y), (x + w, y + h), color, 3)
        cv2.putText(output, label, (x, max(y - 25, 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        
    

    summary = f"Total: {total_cardamom}, Bad: {bad_quality_cardamom}, Good: {total_cardamom - bad_quality_cardamom}"
    cv2.putText(output, summary, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    print(summary)

    cv2.imshow('Final Result with Bounding Boxes', resize_image(output, resize_scale))
    cv2.imshow("Cardamom Mask", resize_image(mask, resize_scale))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Load the reference contour for cardamom
reference_image_path = r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Phase 2\Cardamom\IMG_20241102_130553_0.jpg"
ref_contour = load_reference_contour(reference_image_path)

# Detect cardamom if the reference contour is loaded
if ref_contour is not None:
    image_path = r"C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Phase 2\Cardamom\IMG_20241102_130553_0.jpg"
    detect_cardamom(image_path, ref_contour)

Adaptive HSV Range: Lower=[ 0  0 62], Upper=[ 98 138 255]
Number of contours found: 5
Contour 0 - Area: 91526.5
Contour 0 - Aspect Ratio: 0.45740740740740743
Contour 0 - Match Score: 1.7976931348623157e+308
Contour 0 - HSV: H:27.4 S:44.6 V:123.6
Contour 0 - Edge Ratio: 0.09
Contour 0 - Crack Ratio: 0.00
Contour 0 - Brown/Yellow: False, Has Cracks: False
Contour 0 - Labeled as: Good Cardamom
Contour 1 - Area: 66981.5
Contour 1 - Aspect Ratio: 1.467680608365019
Contour 1 - Match Score: 1.7976931348623157e+308
Contour 1 - HSV: H:20.7 S:65.7 V:123.5
Contour 1 - Edge Ratio: 0.09
Contour 1 - Crack Ratio: 0.00
Contour 1 - Brown/Yellow: False, Has Cracks: False
Contour 1 - Labeled as: Good Cardamom
Contour 2 - Area: 101631.5
Contour 2 - Aspect Ratio: 1.2318435754189945
Contour 2 - Match Score: 1.7976931348623157e+308
Contour 2 - HSV: H:38.6 S:46.7 V:145.0
Contour 2 - Edge Ratio: 0.14
Contour 2 - Crack Ratio: 0.02
Contour 2 - Brown/Yellow: False, Has Cracks: True
Contour 2 - Labeled as: Bad Car

turmeric

In [7]:
import cv2
import numpy as np
import imutils


TURMERIC_MIN_ASPECT_RATIO = 0.2  
TURMERIC_MAX_ASPECT_RATIO = 5.0
TURMERIC_MIN_AREA = 1000
AFFECTED_AREA_THRESHOLD = 0.2

def resize_image(image, width, height):
    """Resize image for display purposes."""
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def detect_turmeric_quality(image_path):
    """Detect turmeric quality, calculate dimensions, and grade the results."""
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Input image not found or failed to load.")
        return
    print("Input image loaded successfully.")

    cv2.imshow("Original Image", resize_image(image, 800, 600))

    alpha = 2.0
    beta = 0
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    cv2.imshow("Brightness and Contrast Adjusted", resize_image(adjusted_image, 800, 600))

    hsv = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)

    lower_turmeric = np.array([20, 50, 50])
    upper_turmeric = np.array([35, 255, 255]) 
    turmeric_mask = cv2.inRange(hsv, lower_turmeric, upper_turmeric)
    cv2.imshow("Turmeric Mask", resize_image(turmeric_mask, 800, 600))

    kernel = np.ones((5, 5), np.uint8)
    turmeric_mask = cv2.morphologyEx(turmeric_mask, cv2.MORPH_CLOSE, kernel)
    turmeric_mask = cv2.morphologyEx(turmeric_mask, cv2.MORPH_OPEN, kernel)
    cv2.imshow("Morphological Cleaned Mask", resize_image(turmeric_mask, 800, 600))

    edges = cv2.Canny(adjusted_image, 100, 200)
    cv2.imshow("Edge Detection", resize_image(edges, 800, 600))

    cnts = cv2.findContours(turmeric_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    output_image = adjusted_image.copy()

    lower_affected = np.array([0, 50, 50])  # Adjusted lower bound for defects
    upper_affected = np.array([20, 255, 255])  # Adjusted upper bound for defects
    mask_affected = cv2.inRange(hsv, lower_affected, upper_affected)
    cv2.imshow("Affected Mask", resize_image(mask_affected, 800, 600))

    total_turmeric = 0
    bad_quality_turmeric = 0

    contour_image = adjusted_image.copy()
    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > TURMERIC_MIN_AREA:
            cv2.drawContours(contour_image, [c], -1, (255, 0, 0), 2)

    cv2.imshow("Contours Before Final Processing", resize_image(contour_image, 800, 600))
    cv2.waitKey(0)

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > TURMERIC_MIN_AREA:
            x, y, w, h = cv2.boundingRect(c)
            aspect_ratio = float(h) / w if w > 0 else 0

            if TURMERIC_MIN_ASPECT_RATIO <= aspect_ratio <= TURMERIC_MAX_ASPECT_RATIO:
                total_turmeric += 1

                turmeric_length = h
                turmeric_width = w

                single_turmeric_mask = np.zeros(turmeric_mask.shape, dtype=np.uint8)
                cv2.drawContours(single_turmeric_mask, [c], -1, 255, thickness=cv2.FILLED)

                affected_turmeric = cv2.bitwise_and(mask_affected, mask_affected, mask=single_turmeric_mask)

                affected_area = cv2.countNonZero(affected_turmeric)
                affected_ratio = affected_area / area if area > 0 else 0

                if affected_ratio > AFFECTED_AREA_THRESHOLD:
                    bad_quality_turmeric += 1
                    label = "Bad Quality Turmeric"
                    color = (0, 0, 255)
                else:
                    label = "Good Quality Turmeric"
                    color = (0, 255, 0)

                cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                cv2.putText(output_image, f"Length: {round(turmeric_length * 0.0264583333)}cm", 
                            (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                cv2.putText(output_image, f"Width: {round(turmeric_width * 0.0264583333)}cm", 
                            (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

                cv2.drawContours(output_image, [c], -1, (255, 0, 0), 2)

    print(f"Total Turmeric Pieces: {total_turmeric}")
    print(f"Bad Quality Turmeric: {bad_quality_turmeric}")
    print(f"Good Quality Turmeric: {total_turmeric - bad_quality_turmeric}")

    grading_text = f"Total: {total_turmeric}, Bad: {bad_quality_turmeric}, Good: {total_turmeric - bad_quality_turmeric}"
    cv2.putText(output_image, grading_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

    cv2.imshow("Final Output with Contours and Bounding Boxes", resize_image(output_image, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

image_path = r'C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\Turmeric\20250124_144945.jpg'
detect_turmeric_quality(image_path)


Input image loaded successfully.
Total Turmeric Pieces: 8
Bad Quality Turmeric: 0
Good Quality Turmeric: 8


clove

In [8]:
import cv2
import numpy as np
import imutils

CLOVE_MIN_AREA = 500
AFFECTED_AREA_THRESHOLD = 0.3

GOOD_CLOVE_COLOR_LOWER = np.array([10, 40, 40])
GOOD_CLOVE_COLOR_UPPER = np.array([30, 255, 180])

BAD_CLOVE_COLOR_LOWER = np.array([35, 50, 50])
BAD_CLOVE_COLOR_UPPER = np.array([90, 255, 200])

def resize_spice_image(image, width, height):  # Rename function
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)


def detect_clove_quality(image_path, reference_path):
    image = cv2.imread(image_path)
    reference_image = cv2.imread(reference_path, 0)
    if image is None or reference_image is None:
        print("Error: Input or reference image not found.")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    edges = cv2.Canny(gray, 50, 150)
    reference_edges = cv2.Canny(reference_image, 50, 150)
    combined_edges = cv2.bitwise_or(edges, reference_edges)

    kernel = np.ones((5, 5), np.uint8)
    combined_edges = cv2.morphologyEx(combined_edges, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(combined_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    output_image = image.copy()

    total_cloves = 0
    bad_quality_cloves = 0

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > CLOVE_MIN_AREA:
            total_cloves += 1
            single_clove_mask = np.zeros(gray.shape, dtype=np.uint8)
            cv2.drawContours(single_clove_mask, [c], -1, 255, thickness=cv2.FILLED)

            clove_hsv = cv2.bitwise_and(hsv, hsv, mask=single_clove_mask)
            good_clove_mask = cv2.inRange(clove_hsv, GOOD_CLOVE_COLOR_LOWER, GOOD_CLOVE_COLOR_UPPER)
            bad_clove_mask = cv2.inRange(clove_hsv, BAD_CLOVE_COLOR_LOWER, BAD_CLOVE_COLOR_UPPER)

            good_area = cv2.countNonZero(good_clove_mask)
            bad_area = cv2.countNonZero(bad_clove_mask)
            total_area = good_area + bad_area

            affected_ratio = bad_area / total_area if total_area > 0 else 0
            print(f"Clove {i}: Good Area = {good_area}, Bad Area = {bad_area}, Affected Ratio = {affected_ratio:.2f}")

            if affected_ratio > AFFECTED_AREA_THRESHOLD:
                bad_quality_cloves += 1
                label = "Bad Quality"
                color = (0, 0, 255)
            else:
                label = "Good Quality"
                color = (0, 255, 0)

            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Debugging: Visualize masks
            cv2.imshow(f"Good Clove Mask {i}", good_clove_mask)
            cv2.imshow(f"Bad Clove Mask {i}", bad_clove_mask)

    print(f"Total Cloves: {total_cloves}")
    print(f"Bad Quality Cloves: {bad_quality_cloves}")
    print(f"Good Quality Cloves: {total_cloves - bad_quality_cloves}")

    grading_text = f"Total: {total_cloves}, Bad: {bad_quality_cloves}, Good: {total_cloves - bad_quality_cloves}"
    cv2.putText(output_image, grading_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow("Final Output", resize_image(output_image, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test paths
image_path = r'C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Phase 2\Clove-L\train\images\asdas_jpeg_jpg.rf.77b2afac4b30adbfb0b7a1d32eb53191.jpg'
reference_path = r'C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Phase 2\Clove-L\train\images\asdas_jpeg_jpg.rf.77b2afac4b30adbfb0b7a1d32eb53191.jpg'
detect_clove_quality(image_path, reference_path)


Clove 2: Good Area = 4481, Bad Area = 771, Affected Ratio = 0.15
Clove 4: Good Area = 3101, Bad Area = 508, Affected Ratio = 0.14
Clove 7: Good Area = 1930, Bad Area = 607, Affected Ratio = 0.24
Clove 8: Good Area = 3156, Bad Area = 608, Affected Ratio = 0.16
Clove 10: Good Area = 2857, Bad Area = 650, Affected Ratio = 0.19
Total Cloves: 5
Bad Quality Cloves: 0
Good Quality Cloves: 5


Combined

In [12]:
import cv2
import numpy as np
import imutils
from sklearn.cluster import KMeans

# Constants for spice detection
SPICE_PARAMETERS = {
    "chili": {
        "aspect_ratio": (0.2, 5.0),
        "min_area": 5000,
        "color_range": ([0, 35, 55], [2, 255, 255]),
    },
    "pepper": {
        "aspect_ratio": (0.8, 1.2),
        "min_area": 300,
        "color_range": ([0, 0, 20], [180, 100, 80]),
    },
    "cardamom": {
        "aspect_ratio": (0.5, 3.0),
        "min_area": 500,
        "color_range": ([25, 40, 30], [50, 255, 255]),  # Green range for cardamom
        "quality": {
            "bad_hue_min": 25,
            "bad_hue_max": 35,
            "bad_sat_min": 50,
            "bad_val_min": 50,
            "crack_val_max": 50,
            "crack_ratio_threshold": 0.01,
        },
    },
    "turmeric": {
        "aspect_ratio": (0.2, 3.0),
        "min_area": 2000,
        "color_range": ([17, 195, 75], [30, 255, 255]),
    },
    "clove": {
        "aspect_ratio": (0.2, 2.5),
        "min_area": 500,
        "color_range": ([0, 0, 10], [10, 180, 85]),
    },
}

resize_scale = 0.5

def resize_image(image, scale):
    width = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def apply_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    return cv2.dilate(edges, kernel, iterations=1)

def clean_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

def compute_similarity(contour, mask):
    mask_contour = np.zeros_like(mask)
    cv2.drawContours(mask_contour, [contour], -1, 255, thickness=cv2.FILLED)
    mask_inside = cv2.bitwise_and(mask, mask_contour)
    overlap = np.count_nonzero(mask_inside)
    total_area = cv2.contourArea(contour)
    return overlap / total_area if total_area > 0 else 0

def draw_text_with_background(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=3, text_color=(255, 255, 255), bg_color=(0, 0, 0), thickness=3):
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_w, text_h = text_size
    x, y = position
    cv2.rectangle(image, (x, y - text_h - 5), (x + text_w + 5, y + 5), bg_color, -1)
    cv2.putText(image, text, (x, y), font, font_scale, text_color, thickness)

def extract_and_classify_contours(image, color_masks, edge_mask, spice_params):
    contours, _ = cv2.findContours(edge_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h < 100:  # Minimum size filter
            continue

        best_spice = None
        best_similarity = 0

        for spice, mask in color_masks.items():
            similarity = compute_similarity(contour, mask)
            if similarity > best_similarity:
                best_similarity = similarity
                best_spice = spice

        if best_spice and best_similarity > 0.3:
            params = spice_params[best_spice]
            aspect_ratio = float(w) / h

            if params["aspect_ratio"][0] <= aspect_ratio <= params["aspect_ratio"][1]:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 3)

                # Basic spice label
                text_x, text_y = x + 10, y + 30
                draw_text_with_background(image, best_spice.capitalize(), (text_x, text_y), font_scale=3, thickness=3)

                # Additional quality check for cardamom
                if best_spice == "cardamom":
                    roi = image[y:y+h, x:x+w]
                    roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                    single_cardamom_mask = np.zeros_like(roi_hsv[:, :, 0])
                    cv2.drawContours(single_cardamom_mask, [contour], -1, 255, thickness=cv2.FILLED)
                    mean_hsv = cv2.mean(roi_hsv, mask=single_cardamom_mask)[:3]
                    hue, sat, val = mean_hsv

                    # Crack detection
                    crack_mask = cv2.inRange(roi_hsv, (0, 0, 0), (180, 255, params["quality"]["crack_val_max"]))
                    crack_mask = cv2.bitwise_and(crack_mask, single_cardamom_mask)
                    kernel_dilate = np.ones((3, 3), np.uint8)
                    crack_mask = cv2.dilate(crack_mask, kernel_dilate, iterations=1)
                    crack_pixels = cv2.countNonZero(crack_mask)
                    total_pixels = cv2.countNonZero(single_cardamom_mask)
                    crack_ratio = crack_pixels / total_pixels if total_pixels > 0 else 0

                    # Quality classification
                    is_brown_yellow = (params["quality"]["bad_hue_min"] <= hue <= params["quality"]["bad_hue_max"] and
                                     sat >= params["quality"]["bad_sat_min"] and
                                     val >= params["quality"]["bad_val_min"])
                    has_cracks = crack_ratio > params["quality"]["crack_ratio_threshold"]
                    quality = "Bad" if is_brown_yellow or has_cracks else "Good"

                    # Add quality label below spice name
                    quality_text_y = y + 60
                    draw_text_with_background(image, f"Quality: {quality}", (text_x, quality_text_y), font_scale=3, thickness=3)

                    print(f"Cardamom {x},{y} - HSV: H:{hue:.1f} S:{sat:.1f} V:{val:.1f}")
                    print(f"Cardamom {x},{y} - Brown/Yellow: {is_brown_yellow}, Has Cracks: {has_cracks}")
                    print(f"Cardamom {x},{y} - Labeled as: {quality}")

    return image

def test_all_spices(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    edge_mask = apply_edge_detection(image)

    color_masks = {}
    for spice, params in SPICE_PARAMETERS.items():
        lower_color = np.array(params["color_range"][0])
        upper_color = np.array(params["color_range"][1])
        mask = cv2.inRange(hsv, lower_color, upper_color)
        mask = clean_mask(mask)
        color_masks[spice] = mask
        cv2.imshow(f"Mask - {spice.capitalize()}", cv2.resize(mask, (400, 300)))

    result_image = extract_and_classify_contours(image.copy(), color_masks, edge_mask, SPICE_PARAMETERS)

    cv2.imshow("Final Classification", cv2.resize(result_image, (800, 600)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the input image
image_path = r"C:\Users\vidya\Downloads\IMG_20250216_202121_2(1).jpg"
test_all_spices(image_path)

Cardamom 515,1658 - HSV: H:0.0 S:0.0 V:0.0
Cardamom 515,1658 - Brown/Yellow: False, Has Cracks: False
Cardamom 515,1658 - Labeled as: Good
Cardamom 2959,1014 - HSV: H:0.0 S:0.0 V:0.0
Cardamom 2959,1014 - Brown/Yellow: False, Has Cracks: False
Cardamom 2959,1014 - Labeled as: Good
